## Using PyMOL for alignment and RMSD calculation of enzyme-complex structures from docking experiments

#### Step 1: Import Pymol python package
Firstly, you need to make sure the PyMOL package is installed. If not, before starting, in the Anaconda prompt type: 'conda install -c schrodinger pymol' to install it. 

The below code was simplified from a CompBioMed e-seminar in which opening PyMOL was demonstrated: “Molecular Visualisation using PyMOL” (https://www.compbiomed.eu/compbiomed-e-seminar-14/). Their code opened the PyMOL GUI, but this component was removed once I knew that the code was working.


In [ ]:
import pymol 
from pymol import cmd

#### Step 2: Load the pdb files for all protein structures
The glob module finds all the pathnames matching a specified pattern according to the rules of the Unix shell - in this case *.pdb.

The absolute file path for where the files are stored on my computer is included for ease for me of copying and pasting it. You will need to input the pathname to where the files are downloaded. Ensure this is absolutely correct, otherwise step 3 will not work.

cmd.load() is the pymol command to load .pdb files into PyMOL

As the PyMOL GUI is not being used it will give the following message: 'PyMOL not running, entering library mode (experimental)'. This is expected.

In [ ]:
import glob 

file_path = input('Please copy and paste absolute file path for .pdb protein files e.g. C:/Users/ew17435/OneDrive - University of Bristol/Documents/SWBio PhD/Year 1/Data Science and Machine Learning/Assessment/SWBio_Assessment/Sample Data/')

file_list = glob.glob(file_path + '*.pdb')

for protein in file_list:
    cmd.load(protein) 

print("(loading .pdb files complete)")

#### Step 3:  Align actKR of each HADDOCK structure to the actKR of M14
The function align_all_to_M14actKR is defined and then run. 

cmd.align() is the PyMOL align command.

The target needs to be specified when the function is recalled. This allows the target to be changed. For example, to another model from BUDE docking called M17.

In [ ]:
def align_all_to_targetactKR(target, mobile_selection='c. B and n. CA+C+N+O', target_selection='c. B and n. CA+C+N+O'):
    
    object_list = cmd.get_names()
    object_list.remove(target)

    for object in range(len(object_list)):
        align = cmd.align('%s & %s'%(object_list[object],mobile_selection),'%s & %s'%(target,target_selection))
        
    
align_all_to_targetactKR('M14')

print("(alignment to M14 actKR complete)")

#### Step 4: Calculate RMSD for actACP between HADDOCK structures and M14

This is the code in which I define my own RMSD function using numpy arrays - the preferred approach which is included in the script:

In [ ]:
from pymol import stored
from pymol import selector

def get_xyz( UserSelection ):  
    
    stored.BackboneAtoms = []
    
    userSelection = UserSelection + ' and c. A and n. CA+C+N+O' #initially had 'and c. A' which didn't work as there wasn't a space before the 'and'
    
    cmd.iterate_state(1, selector.process(userSelection), "stored.BackboneAtoms.append([x,y,z])")
    
    return stored.BackboneAtoms


import numpy as np
      
def calculate_rmsd( target ):
    M14_xyz = get_xyz( target )

    object_list = cmd.get_names()
    object_list.remove(target)

    rmsd_list = [] 
    for protein in object_list:
            xyz = get_xyz(protein)
            rmsd = np.sqrt(np.sum((np.array(M14_xyz)-np.array(xyz))**2)/len(xyz))  
            rmsd_list.append((protein, rmsd))

    rmsd_list_sorted = sorted(rmsd_list,key=lambda x: x[1])
    
    print('RMSD of', target, 'actACP onto:')
    for x in rmsd_list_sorted:
        print(x[0], 'actACP =', x[1])
    
    
calculate_rmsd('M14')


print("Analysis complete")

This is the original code for the calculate_rmsd function which uses nested lists, rather than numpy arrays. 

In [ ]:
from pymol import stored
from pymol import selector

def get_xyz( UserSelection ):  
    
    stored.BackboneAtoms = []
    
    userSelection = UserSelection + ' and c. A and n. CA+C+N+O' #initially had 'and c. A' which didn't work as there wasn't a space before the 'and'
    
    cmd.iterate_state(1, selector.process(userSelection), "stored.BackboneAtoms.append([x,y,z])")
    
    return stored.BackboneAtoms


import numpy as np
      

def calculate_rmsd( target ):
    M14_xyz = get_xyz(target)

    object_list = cmd.get_names()
    object_list.remove(target)
    
    calculated_rmsd_list = []

    for protein in object_list:
        xyz = get_xyz(protein)
        sum = 0
        for i in range(len(xyz)):
            sum += (np.square(M14_xyz[i][0] - xyz[i][0]) + np.square(M14_xyz[i][1] - xyz[i][1]) + np.square(M14_xyz[i][2] - xyz[i][2]))
            #it took a while to work out which components of the nested lists were required in the sum calculation.
        calculated_rmsd = np.sqrt(1/len(xyz) * sum)          
        calculated_rmsd_list.append((protein, calculated_rmsd))
    
    calculated_rmsd_list_sorted = sorted(calculated_rmsd_list,key=lambda x: x[1])
   
    print('RMSD of M14 actACP onto:')
    for x in calculated_rmsd_list_sorted:
        print(x[0], 'actACP =', x[1])

calculate_rmsd('M14')
    

This is code which uses the PyMOL cmd.rms_cur command which was the initial approach for RMSD calculation **(FYI only)**:


In [ ]:
def rmsd_actACP(object1='M14', object1_selection='c. A & n. n+ca+c+o', object2_selection='c. A & n. n+ca+c+o'):
    
    object_2 = cmd.get_names()
    object_2.remove('M14')
    
    rmsd_list = [] #originally included this in the for loop which didn't work

    for i in range(len(object_2)):      
        rmsd = cmd.rms_cur('%s & %s'%(object1,object1_selection),'%s & %s'%(object_2[i],object2_selection), matchmaker=-1)
        rmsd_list.append((object_2[i],rmsd))
    
    rmsd_list_sorted = sorted(rmsd_list,key=lambda x: x[1])

    print('RMSD of M14 actACP onto:')
    for x in rmsd_list_sorted:
        print(x[0], 'actACP =', x[1])
                              
                         
rmsd_actACP()